In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

In [2]:
from omni.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

/home/leon/git/leon/omni/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [ ]:
a = torch.randint(0, 50256, (1, 50, 100))

In [13]:
import tokenizers.processors as processors

def _add_bos_token(tokenizer: AutoTokenizer):
    tokenizer._tokenizer.post_processor = processors.Sequence(
        [
            processors.ByteLevel(trim_offsets=False),
            processors.TemplateProcessing(
                single=f"{tokenizer.bos_token}:0 $A:0",
                pair=f"{tokenizer.bos_token}:0 $A:0 {tokenizer.bos_token}:1 $B:1",
                special_tokens=[
                    (tokenizer.bos_token, tokenizer.bos_token_id),
                ],
            ),
        ]
    )
    return tokenizer

In [14]:
tokenizer2 = _add_bos_token(tokenizer) 

In [7]:
torch.log(torch.tensor(50000))

tensor(10.8198)

In [16]:
test = "thjis is a test, this is a test"

output = tokenizer2(test, padding="max_length", max_length=3, truncation=True, return_overflowing_tokens=True)

In [2]:
d_model=128
hidden_dim = 4 * int(2 * d_model / 3)
hidden_dim = 4 * (
    (hidden_dim + 4 - 1) // 4
)

hidden_dim

340

In [6]:
import torch

logits = torch.tensor([1,5,10], dtype=torch.float32)
torch.softmax(logits, dim=0)

tensor([1.2257e-04, 6.6920e-03, 9.9319e-01])

torch.Size([2, 3, 5])

In [10]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'pad_token': '<pad>'}

In [ ]:
tokenizer.encode(x, padding="max_length", max_length=max_length, truncation=True, return_overflowing_tokens=True)

In [ ]:
tokenizer(test, )

In [ ]:
output

In [ ]:
from omni.modules.pos_embeddings import precompute_freqs_cis_real

pos_embeddings = precompute_freqs_cis_real(64, 512)

In [ ]:
base = torch.pow(2.0, torch.tensor(-8/16))

In [7]:
def _tokenize(dataset, tokenizer: AutoTokenizer, num_proc):
    print("Tokenizing dataset...")
    dataset = dataset.map(
        lambda x: tokenizer(
            x["text"],
            truncation=True,
            max_length=512,
            padding="max_length",
            return_overflowing_tokens=True,
        ),
        batched=True,
        num_proc=num_proc,
        remove_columns=dataset.column_names,
    )
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    return dataset

In [4]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories", split="train", num_proc=1)

In [8]:
_tokenize(ds, tokenizer, 2)

Tokenizing dataset...


Map (num_proc=2):   0%|          | 0/2119719 [00:00<?, ? examples/s]Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/leon/git/leon/omni/.venv/lib/python3.12/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/home/leon/git/leon/omni/.venv/lib/python3.12/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leon/git/leon/omni/.venv/lib/python3.12/site-packages/multiprocess/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/home/leon/git/leon/omni/.venv/lib/python3.12/site-packages/multiprocess/queues.py", line 387, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leon/git/leon/omni/.venv/lib/python3.12/site-packages/dill/_dill.py", line 303, in loads
    return load(file, ignore, **kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "

RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.